<a href="https://colab.research.google.com/github/UtkarshAIITB/DAV-Team-Assignment/blob/main/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the data

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DAV_assignment/comment-toxicity-classification-ques /train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train.shape

(159571, 8)

## Analysing the data

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [ ]:
train.describe()
# low mean indicated, very less comments might be labelled

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
for columns in train.columns[2:]:
  print(columns + ' : count : ' + str(train[columns].sum()))

toxic : count : 15294
severe_toxic : count : 1595
obscene : count : 8449
threat : count : 478
insult : count : 7877
identity_hate : count : 1405


The count values depict that out of total training set of 1,59,571 rows, very less of them are actually labelled in any of the categories.

In [ ]:
new = train.drop(columns = 'id')
new.head(3)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [ ]:
Y = new.iloc[:,1:]

In [ ]:
Y.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [ ]:
# X = new.iloc[:,0]
# X.head()

In [ ]:
import tensorflow as tf

In [ ]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# corpus = []
# for i in range(0, len(X)):
#     # print(i)
#     review = re.sub('[^a-zA-Z]', ' ', X[i])
#     review = review.lower()
#     review = review.split()
    
#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

In [ ]:
import sys
print(sys.getrecursionlimit())

5000


In [ ]:
sys.setrecursionlimit(5000)

In [ ]:
def clean_text(df):
    all_reviews = list()
    lines = df["comment_text"].values.tolist()
    for text in lines:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

In [ ]:
all_comments = clean_text(train)

In [ ]:
all_comments[0:20]

['explan edit made usernam hardcor metallica fan revert nt vandal closur ga vote new york doll fac pleas nt remov templat talk page sinc retir',
 'daww match background colour seemingli stuck thank talk januari utc',
 'hey man realli not tri edit war guy constantli remov relev inform talk edit instead talk page seem care format actual info',
 'ca nt make real suggest improv wonder section statist later subsect type accid think refer may need tidi exact format ie date format etc later noon els first prefer format style refer want pleas let know appear backlog articl review guess may delay review turn list relev form eg wikipediagoodarticlenominationstransport',
 'sir hero chanc rememb page',
 'congratul well use tool well talk',
 'cocksuck piss around work',
 'vandal matt shirvington articl revert pleas nt ban',
 'sorri word nonsens offens anyway not intend write anyth articlewow would jump vandal mere request encycloped one use school refer select breed page almost stub point anim bree

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
TV = TfidfVectorizer(min_df=2, max_features = 1000)   
X = TV.fit_transform(all_comments).toarray()
# y = data.as_matrix(["Review_class"])

In [ ]:
# X[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
RF = RandomForestClassifier()
model1 = MultiOutputClassifier(estimator= RF)
model1.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [ ]:
list3 = Y.columns

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
pred = model1.predict(X_train)
print("Training Results")
for i in range(len(list3)):
    acc1 = accuracy_score(y_train.iloc[:, i], pred[:,i])
    print(list3[i] +':'+ str(acc1))

Training Results
toxic:0.9940700006266843
severe_toxic:0.998284451964655
obscene:0.9971172526164066
threat:0.9996239894717052
insult:0.995832549978066
identity_hate:0.9990129723632262


In [ ]:
pred1 = model1.predict(X_test)
print("Testing Results")
for i in range(len(list3)):
    acc2 = accuracy_score(y_test.iloc[:, i], pred1[:,i])
    print(list3[i]+ ': '+ str(acc2) )

Testing Results
toxic: 0.9520288265705781
severe_toxic: 0.9900360332132226
obscene: 0.9763120789597368
threat: 0.9976813410621964
insult: 0.9668807770640765
identity_hate: 0.9916026946576845


In [ ]:
len(pred1)

31915

In [ ]:
# for i in range (0, len(pred1)):
#   print(pred1[i])

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DAV_assignment/comment-toxicity-classification-ques /test.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
test_comments = clean_text(test)
test_comments[0:20]

['yo bitch ja rule succes ever what hate sad mofuckasi bitch slap ur pethed white face get kiss ass guy sicken ja rule pride da music man dont diss shit nothin wrong bein like tupac brother toofuckin white boy get thing right next time',
 'rfc titl fine imo',
 'sourc zaw ashton lapland',
 'look back sourc inform updat correct form guess sourc nt updat shall updat inform thank messag',
 'nt anonym edit articl',
 'thank understand think highli would not revert without discuss',
 'pleas not add nonsens wikipedia edit consid vandal quickli undon would like experi pleas use sandbox instead thank',
 'dear god site horribl',
 'fool believ number correct number lie ponder number care error persist long time continu reproduc latest reproduct know encyclopædia britannica almanac wich state magnittud fair enough victim today not lot guess peopl come number impress enough nt know know shameless lucki number throw air gc',
 'doubl redirect fix doubl redirect nt blank outer one need edit point final

In [ ]:
X_t = TV.fit_transform(test_comments).toarray()

In [ ]:
pred_final = model1.predict(X_t)

In [ ]:
pred_final[20]

array([0, 0, 0, 0, 0, 0])

In [ ]:
len(pred_final)

159571

In [ ]:
# for i in range(0, len(pred_final)):
#   print(pred_final[i])

In [ ]:
test_ans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DAV_assignment/comment-toxicity-classification-ques /test_labels.csv')
test_ans.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [ ]:
# test_ans.info()
test_ans = test_ans.drop(columns = 'id')
test_ans.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1


In [ ]:
# test_ans = np.where(test_ans == -1, 1, test_ans)
test_ans = test_ans.replace(-1, 1)
test_ans.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,1,1,1,1,1
1,1,1,1,1,1,1
2,1,1,1,1,1,1
3,1,1,1,1,1,1
4,1,1,1,1,1,1


In [ ]:
# test_ans

In [ ]:
for i in range(len(list3)):
    acc3 = accuracy_score(test_ans.iloc[:, i], pred_final[:,i])
    print(list3[i]+ ': '+ str(acc3) )

toxic: 0.4255960930767021
severe_toxic: 0.41532605573111175
obscene: 0.41902797001906456
threat: 0.41634457183150086
insult: 0.4150910135540989
identity_hate: 0.41364158679585283


There is a lot of bias related to non-toxic statements which is reulting in poor performance.